In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from bs4 import BeautifulSoup
import requests
import time

In [54]:
start_year = 2010
end_year = 2019
df_combined_matches = pd.DataFrame()

for year in range(start_year, end_year + 1):
    url = "https://www.footywire.com/afl/footy/ft_match_list?year={}".format(year)
    res = res = requests.get(url)
    soup = BeautifulSoup(res.text, features='lxml')
    tables = soup.select(".data")
    
    years = []
    dates = []
    teams = []
    venues = []
    crowds = []
    results = []
    match_ids = []
    disposals = []
    goals = []

    counter = 0
    is_bye = False
    
    for cell in tables:
        if len(cell.text) <= 1:
            is_bye = True
        if counter == 0 and is_bye == False:
            years.append(year)
            dates.append(cell.text)
        elif counter == 1 and is_bye == False:
            teams.append(cell.text)
        elif counter == 2 and is_bye == False:
            venues.append(cell.text)
        elif counter == 3 and is_bye == False:
            crowds.append(cell.text)
        elif counter == 4 and is_bye == False:
            results.append(cell.text)
            for a in cell.find_all('a', href=True):
                match_ids.append(a['href'])
        elif counter == 5 and is_bye == False:
            disposals.append(cell.text)
        elif counter == 6 and is_bye == False:
            goals.append(cell.text)
        counter += 1
        if counter == 7:
            counter = 0
            is_bye = False

    df_footywire_matches = pd.DataFrame(
        list(
            zip(
                years,
                dates,
                teams,
                venues,
                crowds,
                results,
                match_ids,
                disposals,
                goals,
            )
        ),
        columns=[
            'year',
            'date',
            'teams',
            'venue',
            'crowd',
            'results',
            'match_id',
            'disposals',
            'goals'
        ])

    split_teams = df_footywire_matches["teams"].str.split("\nv", n = 1, expand = True) 
    df_footywire_matches["home_team"]= split_teams[0] 
    df_footywire_matches["away_team"]= split_teams[1] 
    split_results = df_footywire_matches["results"].str.split("-", n = 1, expand = True) 
    df_footywire_matches["home_team_score"]= split_results[0] 
    df_footywire_matches["away_team_score"]= split_results[1] 
    df_footywire_matches.drop(columns =["teams", "results", "disposals", "goals"], inplace = True) 

    df_footywire_matches['home_team'] = df_footywire_matches['home_team'].str.replace('\n', '')
    df_footywire_matches['away_team'] = df_footywire_matches['away_team'].str.replace('\n', '')
    df_footywire_matches['match_id'] = df_footywire_matches['match_id'].str.replace('ft_match_statistics\?mid=', '')
    df_footywire_matches['match_id'] = df_footywire_matches['match_id'].astype(int)
    df_combined_matches = pd.concat([df_combined_matches ,df_footywire_matches])

df_combined_matches

,year,date,venue,crowd,match_id,home_team,away_team,home_team_score,away_team_score
0,2010,Thu 25 Mar 7:10pm,MCG,72010,5089,Richmond,Carlton,64,120
1,2010,Fri 26 Mar 7:40pm,MCG,57772,5090,Geelong,Essendon,125,94
2,2010,Sat 27 Mar 2:10pm,MCG,45615,5091,Melbourne,Hawthorn,61,117
3,2010,Sat 27 Mar 7:10pm,ANZ Stadium,31330,5093,Sydney,St Kilda,88,96
4,2010,Sat 27 Mar 7:30pm,Gabba,29201,5092,Brisbane,West Coast,114,82
...,...,...,...,...,...,...,...,...,...
202,2019,Fri 13 Sep 7:50pm,MCG,51813,9923,Geelong,West Coast,88,68
203,2019,Sat 14 Sep 7:25pm,Gabba,30034,9924,Brisbane,GWS,80,83
204,2019,Fri 20 Sep 7:50pm,MCG,94423,9925,Richmond,Geelong,85,66
205,2019,Sat 21 Sep 4:35pm,MCG,77828,9926,Collingwood,GWS,52,56


In [55]:
timestr = time.strftime("%Y%m%d-%H%M%S")
df_combined_matches.to_csv("Outputs/footywire_team_matches_{}.csv".format(timestr))